# Monai test.

### Setup.

In [ ]:
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
from IPython.display import clear_output
from monai.bundle import ConfigParser, download
from monai.transforms import LoadImage, LoadImaged, Orientation, Orientationd, EnsureChannelFirst, EnsureChannelFirstd, Compose
from tcia_utils import nbia

# Consts.
model_name = "wholeBody_ct_segmentation"
download_data = False
download_model = False
data_dir = "./data"
# dicom_dir = os.path.join(data_dir, "1.3.6.1.4.1.14519.5.2.1.3320.3273.193828570195012288011029757668")
dicom_dir = "c:/users/andrej/desktop/dicom1.nii.gz"
model_path = os.path.join(data_dir, model_name, "models", "model.pt")
config_path = os.path.join(data_dir, model_name, "configs", "inference.json")
simulation_batch_size = None

# Download CT data.
if download_data:
    print("Downloading data")
    cart_name = "nbia-56561691129779503"
    cart_data = nbia.getSharedCart(cart_name)
    df = nbia.downloadSeries(cart_data, format="df", path=data_dir)

# Download the model.
if download_model:
    print("Downloading model")
    download(name=model_name, bundle_dir=data_dir)

# Get config.
# Note: This block of code must occur after the model download block, otherwise config file will not be found.
config = ConfigParser()
config.read_config(config_path)

# Load the model.
print("Loading model")
torch.cuda.empty_cache()
model = config.get_parsed_content("network")
model.load_state_dict(torch.load(model_path))
model.cuda()
model.eval()

# Inferer.
inferer = config.get_parsed_content("inferer")

# Preprocessor.
preprocessing = config.get_parsed_content("preprocessing")

# Postprocessor.
postprocessing = config.get_parsed_content("postprocessing")

### Get the data.

In [ ]:
# Preprocess data.
data = preprocessing({"image": dicom_dir})
data["image"] = data["image"].unsqueeze(0).cuda()  # Add batch dimension and move to GPU.

# Simulate larger batch size.
if simulation_batch_size is not None:
    data["image"] = data["image"].repeat(simulation_batch_size, 1, 1, 1, 1)

print(f"Image shape: {data['image'].shape}")
print("Image meta data:")
print(data["image"].meta)


### Run segmentation.

In [ ]:
with torch.no_grad():
    data["pred"] = inferer(data["image"], network=model)

print(f"Prediction shape: {data['pred'].shape}")

### Postprocessing

In [ ]:
# Remove batch dimension and move to CPU.
data["image"] = data["image"][0].cpu()
data["pred"] = data["pred"][0].cpu()

# Postprocessing.
data = postprocessing(data)

# Get segmentation.
segmentation = data["pred"][0]
segmentation = segmentation.numpy()

### Determine body part

In [ ]:
labels, counts = np.unique(segmentation, return_counts=True)
labels = labels.astype(int)
non_zero_labels = labels != 0
labels = labels[non_zero_labels]
counts = counts[non_zero_labels]
top_label = labels[np.argmax(counts)]

print(f"All labels found: {labels}")
print(f"Label counts: {counts}")
print(f"Top label: {top_label}")

labels_distribution = {}
num_slices = segmentation.shape[2]

for slice_index in range(num_slices):
    labels, counts = np.unique(segmentation[:, :, slice_index], return_counts=True)
    labels = labels.astype(int)    
    non_zero_labels = labels != 0
    labels = labels[non_zero_labels]
    counts = counts[non_zero_labels]
    labels_distribution[slice_index] = {label: count for label, count in zip(labels, counts)}

print("Labels distribution:")
print(labels_distribution)

### Show results.

In [ ]:
num_slices = segmentation.shape[2]

for slice_index in range(num_slices):
    print(f"Showing slice {slice_index + 1}/{num_slices}")
    
    slice = data["image"][0, :, :, slice_index].numpy()
    segmentation_slice = segmentation[:, :, slice_index]
      
    plt.subplots(1, 2, figsize=(12, 6))
    plt.subplot(121)
    plt.pcolormesh(slice.T, cmap="Greys_r")
    plt.axis("off")
    plt.subplot(122)
    plt.pcolormesh(segmentation_slice.T, cmap="nipy_spectral")
    plt.axis("off")
    plt.show()
    
    key = input("Press any key to continue or 'q' to quit")
    if key == "q":
        break
    
    clear_output(wait=True)